In [4]:
import pandas as pd
import plotly.express as px
from datetime import datetime,timedelta,date
import plotly.graph_objects as go

In [25]:
Luxembourg_read = pd.read_csv("C:/Users/j/Desktop/Daten/Daten/Luxembourg.csv",sep= ";", parse_dates=['DAY'])
Luxembourg_normalisiert = Luxembourg_read[["GRID_NO","LATITUDE","LONGITUDE","ALTITUDE"]].drop_duplicates()
Luxembourg_pd = Luxembourg_read[["GRID_NO","DAY","TEMPERATURE_MAX"]]

In [6]:
#Check NA
print(Luxembourg_read.isnull().values.any())

False


In [26]:
def reference_temperatur(latest_day: str,dataframe: pd.DataFrame) -> pd.DataFrame:
    #delete value bevor the latest day
    df_date_cleaned = dataframe[dataframe["DAY"] < latest_day]
    #delete the 29.2
    df_date_cleaned = dataframe[dataframe["DAY"].dt.strftime('%m/%d') != "02/29"]
    df_date_cleaned["DAY"]= df_date_cleaned["DAY"].apply(lambda x: x.replace(year = 2001))

    ts_dates = df_date_cleaned["DAY"].dt.strftime('%m/%d')
    start_time = datetime(year =2001,month =1 , day = 1)
    df_reference = pd.DataFrame()

    for day_loop in range(365):
        start_date = (start_time + timedelta(days = day_loop -15)).strftime('%m/%d')
        end_date= (start_time + timedelta(days = day_loop + 15)).strftime('%m/%d')

        if start_date < "12/17" and end_date > "01/15":
            mask = (ts_dates >= start_date) & (ts_dates <= end_date)
        else:
            mask = (ts_dates >= start_date) | (ts_dates <= end_date)

        saved_df = df_date_cleaned[mask].groupby(by= ["GRID_NO"]).quantile(q=0.9)
        saved_df["DAY"] = (start_time + timedelta(days = day_loop)).strftime('%m/%d')
        
        df_reference = pd.concat([df_reference, saved_df])
    return df_reference.reset_index()

In [27]:
Luxembourg_referenz = reference_temperatur("2010.01.01",Luxembourg_pd)
Luxembourg_referenz.columns = ['reference_temperature' if x=='TEMPERATURE_MAX' else x for x in Luxembourg_referenz.columns]

C:\Users\j\AppData\Local\Temp\ipykernel_19716\3321264957.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
new_df = Luxembourg_pd
new_df = new_df[new_df["DAY"].dt.strftime('%m/%d') != "02/29"]
new_df["DAY"]= new_df["DAY"].apply(lambda x: x.replace(year = 2001)).dt.strftime('%m/%d')

C:\Users\j\AppData\Local\Temp\ipykernel_19716\906092815.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["DAY"]= new_df["DAY"].apply(lambda x: x.replace(year = 2001)).dt.strftime('%m/%d')


In [10]:
fig1 = px.scatter(new_df.loc[new_df["GRID_NO"]== 95097], x="DAY", y="TEMPERATURE_MAX")
fig1.update_traces(marker=dict(color='#369898'))
fig2 = px.line(Luxembourg_referenz.loc[Luxembourg_referenz["GRID_NO"]== 95097], x="DAY", y="reference_temperature")
fig3 = go.Figure(data= fig2.data + fig1.data)
fig3.show()

In [11]:
new_df2 = Luxembourg_pd
new_df2 = new_df2[new_df2["DAY"].dt.strftime('%m/%d') != "02/29"]
new_df2 = new_df2.loc[(new_df2["GRID_NO"] == 95097) & (new_df2["DAY"].dt.year == 1991)]
new_df2["DAY"]= new_df2["DAY"].dt.strftime('%m/%d')

In [12]:
fig1 = px.line(new_df2, x="DAY", y="TEMPERATURE_MAX")
fig1.update_traces(marker=dict(color='#369898'))
fig2 = px.line(Luxembourg_referenz.loc[Luxembourg_referenz["GRID_NO"]== 95097], x="DAY", y="reference_temperature")
fig3 = go.Figure(data= fig2.data + fig1.data)
fig3.show()

In [13]:
def check_reference_temperatur(reference_df, data_df):
    data_df["month_day"] = data_df["DAY"].dt.strftime('%m/%d')
    reference_df = reference_df.rename(columns= {"DAY":"month_day"})
    output_df = pd.merge(data_df,reference_df,on=["GRID_NO","month_day"],how='left')
    return output_df[output_df["TEMPERATURE_MAX"] > output_df["reference_temperature"]].drop("month_day",axis=1)

output=check_reference_temperatur(Luxembourg_referenz,Luxembourg_pd)

C:\Users\j\AppData\Local\Temp\ipykernel_19716\6409738.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
def max_per_year(input_df : pd.DataFrame) -> pd.DataFrame:
    input_df.loc[:,"DAY"]= input_df.loc[:,"DAY"].dt.strftime('%y')
    grouped_df = input_df.groupby(["GRID_NO","DAY"]).max()
    grouped_df_quantile = grouped_df.groupby("GRID_NO").quantile([0.25,0.75]).unstack()
    return grouped_df_quantile.loc[:,"TEMPERATURE_MAX"].reset_index()

max_year_luxembourg =max_per_year(Luxembourg_pd)


C:\Users\j\AppData\Local\Temp\ipykernel_19716\1790791036.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\j\AppData\Local\Temp\ipykernel_19716\1790791036.py:4: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.quantile is deprecated. In a future version, a TypeError will be raised. Before calling .quantile, select only columns which should be valid for the function.



In [15]:
def calculate_magnitude(df_of_max_per_year: pd.DataFrame,reference_df:pd.DataFrame):
    output_df = pd.merge(reference_df,df_of_max_per_year,on=["GRID_NO"],how='left')
    output_df.loc[:,"magnitude"] = (output_df.loc[:,"TEMPERATURE_MAX"] - output_df.loc[:,0.25])/ (output_df.loc[:,0.75]-output_df.loc[:,0.25])

    return output_df[output_df["TEMPERATURE_MAX"]>output_df[0.25]]

In [16]:
single_magnitudes = calculate_magnitude(max_year_luxembourg,output)

In [17]:
def sum_magnitudes(single_magnitudes:pd.DataFrame):
    single_magnitudes.sort_values(by= ["GRID_NO","DAY"])
    single_magnitudes["Start_Date"] = single_magnitudes["DAY"]
    single_magnitudes["End_Date"] = single_magnitudes["DAY"] + timedelta(days = 1)


    output_df = pd.DataFrame()
    new_safe = 0
    for index, row in single_magnitudes.iterrows():
        if new_safe == 1:
            if (row["GRID_NO"] == grid_no) & (end_date == row["Start_Date"]):
                mangnitude += row["magnitude"]
                end_date = row["End_Date"]    
            else:
                df_to_concat = pd.DataFrame({"GRID_NO":grid_no,"Start_Date" : start_date, "End_Date": end_date,"sum_magnitude": mangnitude},index=[0])
                output_df = pd.concat([output_df,df_to_concat])
                new_safe = 0

        if new_safe == 0:
            new_safe = 1
            grid_no = row["GRID_NO"]
            start_date = row["Start_Date"]
            end_date = row["End_Date"]
            mangnitude = row["magnitude"]

    return output_df.reset_index(drop=True)

In [18]:
sum_of_magnitudes = sum_magnitudes(single_magnitudes)
sum_of_magnitudes.head()


,GRID_NO,Start_Date,End_Date,sum_magnitude
0,95097,1982-07-09,1982-07-10,0.102041
1,95097,1983-07-10,1983-07-12,0.122449
2,95097,1983-07-16,1983-07-18,0.503401
3,95097,1983-07-26,1983-07-27,0.537415
4,95097,1983-07-28,1983-07-29,0.047619


In [19]:
single_magnitudes.head()

,GRID_NO,DAY,TEMPERATURE_MAX,reference_temperature,0.25,0.75,magnitude,Start_Date,End_Date
65,95097,1982-07-09,32.0,29.7,31.625,35.3,0.102041,1982-07-09,1982-07-10
97,95097,1983-07-10,31.7,29.8,31.625,35.3,0.020408,1983-07-10,1983-07-11
98,95097,1983-07-11,32.0,29.9,31.625,35.3,0.102041,1983-07-11,1983-07-12
100,95097,1983-07-16,32.0,30.2,31.625,35.3,0.102041,1983-07-16,1983-07-17
101,95097,1983-07-17,33.1,30.3,31.625,35.3,0.401361,1983-07-17,1983-07-18


In [20]:
fig1 = px.scatter(sum_of_magnitudes.loc[sum_of_magnitudes["GRID_NO"]== 95097], x="Start_Date", y="sum_magnitude")
fig1.update_traces(marker=dict(color='#369898'))
fig2 = px.scatter(single_magnitudes.loc[single_magnitudes["GRID_NO"]== 95097], x = "DAY",y = "magnitude")
fig3 = go.Figure(data= fig2.data + fig1.data)
fig3.show()

In [ ]:
schweden = pd.read_csv("C:/Users/j/Desktop/Daten/Daten/Sverige.csv",sep= ";", parse_dates=['DAY'])
schweden_normalisiert = schweden[["GRID_NO","LATITUDE","LONGITUDE","ALTITUDE"]].drop_duplicates()
schweden_pd = schweden[["GRID_NO","DAY","TEMPERATURE_MAX"]]

reference_schweden = reference_temperatur("2010.01.01",schweden_pd)
reference_schweden.columns = ['reference_temperature' if x=='TEMPERATURE_MAX' else x for x in reference_schweden.columns]
reference_new_sweden = check_reference_temperatur(reference_schweden,schweden_pd)
max_year_schweden = max_per_year(schweden_pd)
single_magnitudes_schweden = calculate_magnitude(max_year_schweden,reference_new_sweden)
sum_of_magnitudes_schweden = sum_magnitudes(single_magnitudes_schweden)
output_schweden = pd.merge(sum_of_magnitudes_schweden,schweden_normalisiert,on=["GRID_NO"],how='left')

In [32]:
output_schweden.to_excel("output_sweden.xlsx")